In [38]:
# !pip install --upgrade pip
# !pip install pigeonXT
# !pip install jupyter_innotater

In [10]:
import jupyterannotate
import pigeonXT as pixt
import pandas as pd
import numpy as np


def load_data(type):
    if type == "job":
        df = pd.read_json("../raw/vacancies.json")
        df["full_text"] = df["fulltext"] = df["name"] + "\n" + df["description"]

    elif type == "course":
        df = pd.read_json("../raw/learning_opportunities.json")
        df = df[df["active"] == True]
        keep_ids = {1, 5, 9}
        df = df[df["study_ids"].apply(lambda x: bool(set(x) & keep_ids))]

    return df


def load_lv2():
    df = pd.read_csv("../taxonomy/taxonomy_V4.csv", sep=",")
    df = df.dropna(subset=["Definition", "Type Level 2"])
    # df["name+definition"] = df.apply(concatenate_cols_skillname, axis=1)
    keep_cols = [
        "Type Level 2",
    ]
    df = df[keep_cols]
    df = df.drop_duplicates().reset_index(drop=True)
    list_lv2 = df["Type Level 2"].tolist()
    list_lv2 = list_lv2 + ["NONE", "ADD_NEW"]

    return list_lv2


df = load_data("job")
tax = load_lv2()

# SKILL_LABELS = ["Skill"]
LEVEL_LABELS = ["Beginner", "Intermediate", "Expert", "Unknown"]

DOCUMENTS = list(df.sample(100, random_state=42)["full_text"])

# Instantiate widget
annotation_widget = jupyterannotate.AnnotateWidget(
    docs=DOCUMENTS,
    labels=LEVEL_LABELS,
)

In [4]:
annotation_widget

AnnotateWidget(value=None, docs=['ICT Security Operations Manager (a) 80-100%\nICT Security Operations Manager…

In [39]:
annotation_widget.spans

TO_MATCH = 0

In [15]:
def get_skills_per_doc(doc_num):
    span = annotation_widget.spans[doc_num]
    list_of_skills = []
    for skill in span:
        text = skill["text"]
        text = text.strip()
        list_of_skills.append(text)
    skills = [{"example": skill} for skill in list_of_skills]
    df = pd.DataFrame(skills)
    return df

In [23]:
try:
    SKILLS_DF = get_skills_per_doc(TO_MATCH)
except:
    print("No annotations found for this document")

In [24]:
tax

In [37]:
annotations = pixt.annotate(
    SKILLS_DF,
    options=tax,
    task_type="multilabel-classification",
    buttons_in_a_row=5,
    reset_buttons_after_click=True,
)

HTML(value='0 of 3 Examples annotated, Current Position: 0 ')

Output()